In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly
plotly.offline.init_notebook_mode()
from toolkit.tire_model.tire_model_pacejka_2010 import *
from toolkit.tire_model.tire_model_fitting import *
from toolkit.tire_model.ttc_loader import *
from toolkit.tire_model.tire_fitting_masks import *
from toolkit.tire_model.tire_model_utils import *
from toolkit.common.constants import *

In [ ]:
# Get the ttc data for the tire you want to fit
combi_runs, cornering, drive_brake, name = load_runs(get_R20_18x6_7_runs())
# create all the boring lists and stuff
params_list: List = []
error_list: List = []

In [ ]:
# Let's just take a leap of fath and ignore this for a second, we'll come back to it
tire_model = tire_model_from_arr(H_R20_18X6_7)
fz_nom = tire_model.FNOMIN

Alright say we want to calculate the lateral force from a tire, the tire has a lateral stiffness of $K_{\alpha y}$.<br>
We can calculate the lateral force as follows: <br>
$F_{yp} = F_z * K_{\alpha y} * \alpha_y$ <br>
Lets take a look at some data and see how this fits

In [ ]:
# Lets get a cornering run (where they sweep slip angle) at one pressure, IA, and load
df = remove_time_gaps(filter_vel(filter_press(cornering, press=82700, d_press=4000), 11.1, 0.1))
df = df[np.abs(df.IA) < np.deg2rad(1)]
df = df[np.abs(df.FZ - fz_nom) < fz_nom*0.1+20]
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
# and I know off the top of my head that the range of slip angles is between -0.25 and 0.25 rad
# so lets create a sweep of slip angles
slip_angles = np.linspace(-0.25, 0.25, 100)
# and then lets feed that through out equation to get the forces
kya = tire_model.PKY1 # these are the coefficients in MFTire 6.2 that are equal to the Kay we want if you cancel everything out 
fy = fz_nom * kya * slip_angles
# and then lets plot it
fig = go.Figure()
fig.add_trace(go.Scattergl(x=np.rad2deg(df.SA), y=df.FY, name='TTC data', mode='markers', marker=dict(size=1)))
fig.add_trace(go.Scattergl(x=np.rad2deg(slip_angles), y=fy, name='Our model 1'))
fig.update_layout(template="plotly_dark", title=f"Fy vs SA at {fz_nom} N Fz", xaxis_title="SA (deg)", yaxis_title="Fy (N)")
fig.update_yaxes(range=[-3000, 3000])
fig.show()

Now that is what I would call abysmal, lets improve it by treating this like a portion of a sinusoid and considering our maximum friction coefficient of $\mu_y$. <br>
We can calculate the lateral force as follows: <br>
$D_y = \mu_y * F_z$ <br>
$B_y = \frac{K_{\alpha y}}{D_y}$<br>
$F_{yp} = D_y * sin(arctan( B_y * \alpha_y))$ <br>
We scale down the stiffness by the friction coefficient before we do our trigonometric function, then we scale it back up because we know that our max $\mu_y * F_z$ will be the amplitude of our sinusoid. <br>

In [ ]:
mu_y = tire_model.PDY1 # now we get a coefficient for mu_y
d_y = mu_y * fz_nom
# and for reasons we will get to soon, we are now going to multiply the stiffness by the normal force
kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(1/tire_model.PKY2)) # and we will scale it a bit for now
b_y = kya / d_y # and then we can get b_y
fy_two = d_y * np.sin(np.arctan(b_y * slip_angles))
fig.add_trace(go.Scattergl(x=np.rad2deg(slip_angles), y=fy_two, name='Our model 2'))

Alright so now we are getting somewhere, lets add another factor, $C_y$ so that we can squish the sinusoid in the $\alpha_y$ direction. <br>
$D_y = \mu_y * F_z$ <br>
$B_y = \frac{K_{\alpha y}}{D_y * C_y}$, because we need to normalize $B_y$ still.<br>
$F_{yp} = D_y * sin(C_y * arctan( B_y * \alpha_y))$ <br>
Lets see how this fits

In [ ]:
c_y = tire_model.PCY1 # now we get a coefficient for c_y
d_y = mu_y * fz_nom
b_y = kya / (d_y * c_y) # and then we can get b_y
fy_three = d_y * np.sin(c_y * np.arctan(b_y * slip_angles))
fig.add_trace(go.Scattergl(x=np.rad2deg(slip_angles), y=fy_three, name='Our model 3'))
fig.show()

Alright so now the end behavior is much better, we are really close. <br>
Lets add another factor, $E_y$, so that we can stretch the sinusoid in a nonlinear way in the center <br>
This gets us to Pacejka and his magic formula. <br>
$F_{yp} = D_y * sin(C_y * arctan( B_y * \alpha_y - E_y * (B_y * \alpha_y - arctan(B_y * \alpha_y))))$ <br>
and if you just want to fit a magic formula to one tire, at one pressure, temperature, camber, and load with no combined loading you can use that formula as it is. <br>

In [ ]:
d_y = mu_y * fz_nom
e_y = tire_model.PEY1# now we get a coefficient for e_y
b_y = kya / (d_y * c_y) # and then we can get b_y
fy_four = d_y * np.sin(c_y * np.arctan(b_y * slip_angles - e_y * (b_y * slip_angles - np.arctan(b_y * slip_angles))))
fig.add_trace(go.Scattergl(x=np.rad2deg(slip_angles), y=fy_four, name='Our model 4'))
fig.show()

Now that is super clean fit, and enough by itself to be useful, but lets try and add some more conditions to it. <br>
First we can try different normal loads and see how our fit changes. <br>

In [ ]:
def plot_fit(fz_targ, func, sa, ia, pres, kappa, figs, dg, name, fz_err=False):
    dd = dg[np.abs(dg.FZ - fz_targ) < fz_targ*0.1+20]
    dd = dd[np.abs(dd.IA - ia) < np.deg2rad(0.5)]
    fy = func(fz_targ, sa, ia, pres, kappa, offsets=True)
    fy_real = func(fz_targ, dd.SA, dd.IA, dd.P, dd.SL)
    shy, svy = None, None
    if type(fy) is tuple:
        shy = fy[1]
        svy = fy[2]
        fy = fy[0]
    error = np.mean(np.abs(fy_real - dd.FY))
    figs.add_trace(go.Scattergl(x=np.rad2deg(sa), y=fy, name=f"{name}\t{error:.2f} N rmse", legendgroup=f"{name}"))
    if fz_err:
        fz_err = (dd.FZ - fz_targ)
        figs.add_trace(go.Scattergl(x=np.rad2deg(dd.SA), y=dd.FZ, mode='markers', marker=dict(size=1.5, color=fz_err, colorscale='Viridis', showscale=True), legendgroup=f"{name}", showlegend=False))
    else:
        figs.add_trace(go.Scattergl(x=np.rad2deg(dd.SA), y=dd.FY, mode='markers', marker=dict(size=1), legendgroup=f"{name}", showlegend=False))
    if shy is not None:
        figs.add_trace(go.Scattergl(x=shy, y=svy, mode='markers', marker=dict(size=4, color='red'), legendgroup=f"{name}", showlegend=False))

In [ ]:
df = remove_time_gaps(filter_vel(filter_press(cornering, press=82700, d_press=4000), 11.1, 0.1))
df = df[np.abs(df.IA) < np.deg2rad(1)]
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(1/tire_model.PKY2))
mu_y = tire_model.PDY1
def model(fz_targ, sa, _, press, kappa, offsets=False):
    c_y = tire_model.PCY1 # now we get a coefficient for c_y
    d_y = mu_y * fz_targ
    e_y = tire_model.PEY1# now we get a coefficient for e_y
    b_y = kya / (d_y * c_y) # and then we can get b_y
    fy = d_y * np.sin(c_y * np.arctan(b_y * sa - e_y * (b_y * sa - np.arctan(b_y * sa))))
    return fy
fig = go.Figure()
sweeps = [220, 440, 660, 880, 1100]
for fza in sweeps:
    plot_fit(fza, model, slip_angles, 0.0, 0.0, 0.0, fig, df, f"Fit {fza} N")
fig.update_layout(template="plotly_dark", title=f"Fy vs SA", xaxis_title="SA (deg)", yaxis_title="Fy (N)")
fig.update_yaxes(range=[-3000, 3000])
fig.show()
# And I allways like to look at the histogram of the Fz to see how consistent it is, for this run you can see that it starts to get a little sloppy on the 800 N run and is bad on the 1100 N run
fig = go.Figure()
fig.add_trace(go.Histogram(x=df.FZ, nbinsx=400, histnorm='probability density'))
fig.update_layout(template="plotly_dark", title=f"Histogram of Fz", xaxis_title="Fz (N)", yaxis_title="Count")
fig.show()

So the slope through the origin gets fucked as we deviate from our nominal normal loads <br>
To solve this Pacejka scales $\mu_y$ by the normal load, with the familiar $sin(arctan())$ situation. <br>
$K_{y\alpha} = P_{ky1} * F_{z0} * sin(P_{ky4} * arctan(\frac{F_z}{P_{ky2} * F_{z0}}))$

In [ ]:
df = remove_time_gaps(filter_vel(filter_press(cornering, press=82700, d_press=4000), 11.1, 0.1))
df = df[np.abs(df.IA) < np.deg2rad(1)]
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
mu_y = tire_model.PDY1
def model(fz_targ, sa, _, press, kappa, offsets=True):
    kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(fz_targ/(tire_model.PKY2*fz_nom)))
    c_y = tire_model.PCY1 # now we get a coefficient for c_y
    d_y = mu_y * fz_targ
    e_y = tire_model.PEY1# now we get a coefficient for e_y
    b_y = kya / (d_y * c_y) # and then we can get b_y
    fy = d_y * np.sin(c_y * np.arctan(b_y * sa - e_y * (b_y * sa - np.arctan(b_y * sa))))
    return fy
fig = go.Figure()
for fza in sweeps:
    plot_fit(fza, model, slip_angles, 0.0, 0.0, 0.0, fig, df, f"Fit {fza} N")
fig.update_layout(template="plotly_dark", title=f"Fy vs SA", xaxis_title="SA (deg)", yaxis_title="Fy (N)")
fig.update_yaxes(range=[-3000, 3000])
fig.show()

So as you can see our model underestimates the lateral force for high slip angle at low normal loads and is closer at our highest normal force which also happens to be our nominal <br>
Lets add a load dependence factor to our $\mu_y$ equation so that the max $\mu$ for a given normal load can change <br>
We shall also introduce a new value for $F_z$ normalized by $F_{z0}$, the nominal normal load<br>
$df_z=\frac{F_z-F_{z0}}{F_{z0}}$<br>
$\mu_y=(P_{Dy1}+P_{Dy2}*df_z)$


In [ ]:
df = remove_time_gaps(filter_vel(filter_press(cornering, press=82700, d_press=4000), 11.1, 0.1))
df = df[np.abs(df.IA) < np.deg2rad(1)]
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
def model(fz_targ, sa, _, press, kappa, offsets=True):
    df_z = (fz_targ - fz_nom) / fz_nom
    mu_y = tire_model.PDY1 + tire_model.PDY2 * df_z
    kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(fz_targ/(tire_model.PKY2*fz_nom)))
    c_y = tire_model.PCY1 # now we get a coefficient for c_y
    d_y = mu_y * fz_targ
    e_y = tire_model.PEY1# now we get a coefficient for e_y
    b_y = kya / (d_y * c_y) # and then we can get b_y
    fy = d_y * np.sin(c_y * np.arctan(b_y * sa - e_y * (b_y * sa - np.arctan(b_y * sa))))
    return fy
fig = go.Figure()
for fza in sweeps:
    plot_fit(fza, model, slip_angles, 0.0, 0.0, 0.0, fig, df, f"Fit {fza} N")
fig.update_layout(template="plotly_dark", title=f"Fy vs SA", xaxis_title="SA (deg)", yaxis_title="Fy (N)")
fig.update_yaxes(range=[-3000, 3000])
fig.show()

Well that worked for the highest slip angle at low normal load, most of our error right now is because the tire is asymmetrical, but lets try and give it some load dependent nonlinear stretch along $\alpha$ by adding a load dependence to $E_y$ to soften it out at the ends<br>
$E_y=P_{Ey1} + P_{Ey2}*df_z$

In [ ]:
df = remove_time_gaps(filter_vel(filter_press(cornering, press=82700, d_press=4000), 11.1, 0.1))
df = df[np.abs(df.IA) < np.deg2rad(1)]
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
def model(fz_targ, sa, _, press, kappa, offsets=True):
    df_z = (fz_targ - fz_nom) / fz_nom
    mu_y = tire_model.PDY1 + tire_model.PDY2 * df_z
    kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(fz_targ/(tire_model.PKY2*fz_nom)))
    c_y = tire_model.PCY1 # now we get a coefficient for c_y
    d_y = mu_y * fz_targ
    e_y = tire_model.PEY1 + tire_model.PEY2 * df_z# now we get a coefficient for e_y
    b_y = kya / (d_y * c_y) # and then we can get b_y
    fy = d_y * np.sin(c_y * np.arctan(b_y * sa - e_y * (b_y * sa - np.arctan(b_y * sa))))
    return fy
fig = go.Figure()
for fza in sweeps:
    plot_fit(fza, model, slip_angles, 0.0, 0.0, 0.0, fig, df, f"Fit {fza} N")
fig.update_layout(template="plotly_dark", title=f"Fy vs SA", xaxis_title="SA (deg)", yaxis_title="Fy (N)")
fig.update_yaxes(range=[-3000, 3000])
fig.show()

Alright at the moment our model is symmetrical the tire is not, so lets give the tire a static and load dependent shift in the $\alpha_y$ direction and in the vertical direction<br>
$S_{Hy0}=P_{Hy1}+P_{Hy2}*df_z$<br>
$S_{Vy0}=F_z(P_{Vy1}+P_{Vy2}*df_z)$<br>
$\alpha_y=\alpha_F+S_{Hy0}$<br>
$F_{yp}=D_y*sin(C_y*arctan(B_y*\alpha_y-E_y*(B_y*\alpha_y-arctan(B_y*\alpha_y))))+S_{Vy0}$

In [ ]:
df = remove_time_gaps(filter_vel(filter_press(cornering, press=82700, d_press=6000), 11.1, 0.1))
df = df[np.abs(df.IA) < np.deg2rad(1)]
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
def model(fz_targ, sa, _, press, kappa, offsets=False):
    df_z = (fz_targ - fz_nom) / fz_nom
    mu_y = tire_model.PDY1 + tire_model.PDY2 * df_z
    kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(fz_targ/(tire_model.PKY2*fz_nom)))
    c_y = tire_model.PCY1 # now we get a coefficient for c_y
    d_y = mu_y * fz_targ
    e_y = tire_model.PEY1 + tire_model.PEY2 * df_z# now we get a coefficient for e_y
    b_y = kya / (d_y * c_y) # and then we can get b_y
    shy_0 = tire_model.PHY1 + tire_model.PHY2 * df_z
    svy_0 = (tire_model.PVY1 + tire_model.PVY2 * df_z) * fz_targ
    svy = svy_0
    shy = shy_0
    sa_y = sa + shy
    fy = d_y * np.sin(c_y * np.arctan(b_y * sa_y - e_y * (b_y * sa_y - np.arctan(b_y * sa_y)))) + svy
    if offsets:
        return (fy, np.ones(1) * np.rad2deg(-shy), np.ones(1) * svy)
    return fy
fig = go.Figure()
for fza in sweeps:
    plot_fit(fza, model, slip_angles, 0.0, 0.0, 0.0, fig, df, f"Fit {fza} N")
fig.update_layout(template="plotly_dark", title=f"Fy vs SA", xaxis_title="SA (deg)", yaxis_title="Fy (N)")
fig.update_yaxes(range=[-3000, 3000])
fig.show()

To my understating $S_{Hy0}$ would be primarily ply steer, as it affects the tendency if the strings (as in string model) along the tire to twist to a slip angle, whereas $S_{Vy0}$ is the constant force brought on by the conicity of the tire <br>
Regardless it is still a question to me if these should be included while you fit the model and then set to zero afterward, or just left at zero, as we cannot fully separate ply steer from conicity without more tests <br>

One thing that it is incredibly important to remember is that not all the points in a sweep will have the same normal load, so be careful overfitting <br>
Lets plot that again but color the points by their deviation from the target normal load

In [ ]:
fig = go.Figure()
for fza in sweeps:
    plot_fit(fza, model, slip_angles, 0.0, 0.0, 0.0, fig, df, f"Fit {fza} N", fz_err=True)
fig.update_layout(template="plotly_dark", title=f"Fy vs SA", xaxis_title="SA (deg)", yaxis_title="Fy (N)")
# move the legend to the top left
fig.update_layout(legend=dict(x=0, y=1))
fig.update_yaxes(range=[-3000, 3000])
fig.show()

In this case we can see that wasn't our problem but it is still good to keep in mind <br>

Alright so now we have Pacejka's Magic Formula as it appeared in his 1989 paper, well I mean we jumped the gun with $P_{Ky4}$ and we still haven't added the camber dependence, but we are getting there <br>
Lets plot some runs with camber and see where we stand

In [ ]:
df = remove_time_gaps(filter_vel(filter_press(cornering, press=82700, d_press=6000), 11.1, 0.1))
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
def model(fz_targ, sa, ia_targ, press, kappa, offsets=False):
    df_z = (fz_targ - fz_nom) / fz_nom
    mu_y = tire_model.PDY1 + tire_model.PDY2 * df_z
    kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(fz_targ/(tire_model.PKY2*fz_nom)))
    c_y = tire_model.PCY1 # now we get a coefficient for c_y
    d_y = mu_y * fz_targ
    e_y = tire_model.PEY1 + tire_model.PEY2 * df_z# now we get a coefficient for e_y
    b_y = kya / (d_y * c_y) # and then we can get b_y
    shy_0 = tire_model.PHY1 + tire_model.PHY2 * df_z
    svy_0 = (tire_model.PVY1 + tire_model.PVY2 * df_z) * fz_targ
    svy = svy_0
    shy = shy_0
    sa_y = sa + shy
    fy = d_y * np.sin(c_y * np.arctan(b_y * sa_y - e_y * (b_y * sa_y - np.arctan(b_y * sa_y)))) + svy
    if offsets:
        return (fy, np.ones(1) * np.rad2deg(-shy), np.ones(1) * svy)
    return fy
fig = go.Figure()
for fza in sweeps:
    for iaa in [0, 2, 4]:
        plot_fit(fza, model, slip_angles, np.deg2rad(iaa), 0.0, 0.0, fig, df, f"Fit {fza} N {iaa} deg")
fig.update_layout(template="plotly_dark", title=f"Fy vs SA", xaxis_title="SA (deg)", yaxis_title="Fy (N)")
fig.update_yaxes(range=[-3000, 3000])
fig.show()

Ok so it appears that at low normal loads there isn't much difference at all but at high normal loads the camber shifts the graph, we will add some horizontal and vertical shift that is dependent on load and camber <br>
But lets just start out with the horizontal shift <br>
$K_{y\gamma}=F_z(P_{Ky6}+P_{Ky7}*df_z)$<br>
$S_{Hy0}=P_{Hy1}+P_{Hy2}*df_z$<br>
$S_{Hy\gamma}=\frac{K_{y\gamma}\gamma}{K_{y\alpha}}$<br>
$S_{Hy}=S_{Hy0}+S_{Hy\gamma}$<br>

In [ ]:
df = remove_time_gaps(filter_vel(filter_press(cornering, press=82700, d_press=6000), 11.1, 0.1))
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
def model(fz_targ, sa, ia_targ, press, kappa, offsets=False):
    df_z = (fz_targ - fz_nom) / fz_nom
    mu_y = tire_model.PDY1 + tire_model.PDY2 * df_z
    kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(fz_targ/(tire_model.PKY2*fz_nom)))
    c_y = tire_model.PCY1 # now we get a coefficient for c_y
    d_y = mu_y * fz_targ
    e_y = tire_model.PEY1 + tire_model.PEY2 * df_z# now we get a coefficient for e_y
    b_y = kya / (d_y * c_y) # and then we can get b_y
    shy_0 = tire_model.PHY1 + tire_model.PHY2 * df_z
    svy_0 = (tire_model.PVY1 + tire_model.PVY2 * df_z) * fz_targ
    kyg = (tire_model.PKY6 + tire_model.PKY7 * df_z) * fz_targ
    shy_g = (kyg * ia_targ) / kya
    shy = shy_0 + shy_g
    svy = svy_0
    sa_y = sa + shy
    fy = d_y * np.sin(c_y * np.arctan(b_y * sa_y - e_y * (b_y * sa_y - np.arctan(b_y * sa_y)))) + svy
    if offsets:
        return (fy, np.ones(1) * np.rad2deg(-shy), np.ones(1) * svy)
    return fy
fig = go.Figure()
for fza in sweeps:
    for iaa in [0, 2, 4]:
        plot_fit(fza, model, slip_angles, np.deg2rad(iaa), 0.0, 0.0, fig, df, f"Fit {fza} N {iaa} deg")
fig.update_layout(template="plotly_dark", title=f"Fy vs SA", xaxis_title="SA (deg)", yaxis_title="Fy (N)")
fig.update_yaxes(range=[-3000, 3000])
fig.show()

Alright now at high normal loads the error went down, lets add in vertical shift see if that helps<br>
$S_{Vy0}=F_z(P_{Vy1}+P_{Vy2}*df_z)$<br>
$S_{Vy\gamma}=F_z(P_{Vy3}+P_{Vy4}*df_z)\gamma$<br>
$S_{Vy}=S_{Vy0}+S_{Vy\gamma}$<br>
But we also need to cancel this out for the vertical shift, so we can just subtract it from the horizontal shift <br>
$S_{Hy\gamma}=\frac{K_{y\gamma}\gamma-S_{Vy\gamma}}{K_{y\alpha}}$<br>
$S_{Hy}=S_{Hy0}+S_{Hy\gamma}$<br>

In [ ]:
df = remove_time_gaps(filter_vel(filter_press(cornering, press=82700, d_press=6000), 11.1, 0.1))
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
def model(fz_targ, sa, ia_targ, press, kappa, offsets=False):
    df_z = (fz_targ - fz_nom) / fz_nom
    mu_y = tire_model.PDY1 + tire_model.PDY2 * df_z
    kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(fz_targ/(tire_model.PKY2*fz_nom)))
    c_y = tire_model.PCY1 # now we get a coefficient for c_y
    d_y = mu_y * fz_targ
    e_y = tire_model.PEY1 + tire_model.PEY2 * df_z# now we get a coefficient for e_y
    b_y = kya / (d_y * c_y) # and then we can get b_y
    shy_0 = tire_model.PHY1 + tire_model.PHY2 * df_z
    svy_0 = (tire_model.PVY1 + tire_model.PVY2 * df_z) * fz_targ
    svy_g = (tire_model.PVY3 + tire_model.PVY4 * df_z) * fz_targ * ia_targ
    svy = svy_0 + svy_g
    kyg = (tire_model.PKY6 + tire_model.PKY7 * df_z) * fz_targ
    shy_g = (kyg * ia_targ - svy_g) / kya
    shy = shy_0 + shy_g
    sa_y = sa + shy
    fy = d_y * np.sin(c_y * np.arctan(b_y * sa_y - e_y * (b_y * sa_y - np.arctan(b_y * sa_y)))) + svy
    if offsets:
        return (fy, np.ones(1) * np.rad2deg(-shy), np.ones(1) * svy)
    return fy
fig = go.Figure()
for fza in sweeps:
    for iaa in [0, 2, 4]:
        plot_fit(fza, model, slip_angles, np.deg2rad(iaa), 0.0, 0.0, fig, df, f"Fit {fza} N {iaa} deg")
fig.update_layout(template="plotly_dark", title=f"Fy vs SA", xaxis_title="SA (deg)", yaxis_title="Fy (N)")
fig.update_yaxes(range=[-3000, 3000])
fig.show()

That helped a little bit but nothing dramatic<br>
Now our highest error is at high camber when we are near our nominal Fz, so lets add a camber factor to the $\mu_y$ equation <br>
$\mu_y=(P_{Dy1}+P_{Dy2}*df_z)*(1+P_{Dy3}*\gamma^2)$<br>

In [ ]:
df = remove_time_gaps(filter_vel(filter_press(cornering, press=82700, d_press=6000), 11.1, 0.1))
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
def model(fz_targ, sa, ia_targ, press, kappa, offsets=False):
    df_z = (fz_targ - fz_nom) / fz_nom
    mu_y = (tire_model.PDY1 + tire_model.PDY2 * df_z) * (1 - tire_model.PDY3 * ia_targ**2)
    kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(fz_targ/(tire_model.PKY2*fz_nom)))
    c_y = tire_model.PCY1 # now we get a coefficient for c_y
    d_y = mu_y * fz_targ
    e_y = tire_model.PEY1 + tire_model.PEY2 * df_z# now we get a coefficient for e_y
    b_y = kya / (d_y * c_y) # and then we can get b_y
    shy_0 = tire_model.PHY1 + tire_model.PHY2 * df_z
    svy_0 = (tire_model.PVY1 + tire_model.PVY2 * df_z) * fz_targ
    svy_g = (tire_model.PVY3 + tire_model.PVY4 * df_z) * fz_targ * ia_targ
    svy = svy_0 + svy_g
    kyg = (tire_model.PKY6 + tire_model.PKY7 * df_z) * fz_targ
    shy_g = (kyg * ia_targ - svy_g) / kya
    shy = shy_0 + shy_g
    sa_y = sa + shy
    fy = d_y * np.sin(c_y * np.arctan(b_y * sa_y - e_y * (b_y * sa_y - np.arctan(b_y * sa_y)))) + svy
    if offsets:
        return (fy, np.ones(1) * np.rad2deg(-shy), np.ones(1) * svy)
    return fy
fig = go.Figure()
for fza in sweeps:
    for iaa in [0, 2, 4]:
        plot_fit(fza, model, slip_angles, np.deg2rad(iaa), 0.0, 0.0, fig, df, f"Fit {fza} N {iaa} deg")
fig.update_layout(template="plotly_dark", title=f"Fy vs SA", xaxis_title="SA (deg)", yaxis_title="Fy (N)")
fig.update_yaxes(range=[-3000, 3000])
fig.show()

Lovley, now this is good enough for me, but lets just copy the rest of pacejka's $\gamma$ dependent terms and see what happens <br>
We shall add the wacky $E_y$ camber terms first <br>
$E_y=(P_{Ey1}+P_{Ey2}*df_z)(1+P_{Ey5}*\gamma^2-(P_{Ey3}+P_{Ey4}\gamma)sgn(\alpha))$<br>
It is unclear to me why exactly it is formulated this way as I don't have any experience with radial tires<br>
Also I am a little sus about using $\gamma^2$ terms while we have testing data with only 3 steps of camber, but were way in the weeds here

In [ ]:
df = remove_time_gaps(filter_vel(filter_press(cornering, press=82700, d_press=6000), 11.1, 0.1))
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
def model(fz_targ, sa, ia_targ, press, kappa, offsets=False):
    df_z = (fz_targ - fz_nom) / fz_nom
    mu_y = (tire_model.PDY1 + tire_model.PDY2 * df_z) * (1 - tire_model.PDY3 * ia_targ**2)
    kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(fz_targ/(tire_model.PKY2*fz_nom)))
    c_y = tire_model.PCY1 # now we get a coefficient for c_y
    d_y = mu_y * fz_targ
    e_y = (tire_model.PEY1 + tire_model.PEY2 * df_z) * (1 + (tire_model.PEY5 * ia_targ**2) - (tire_model.PEY3 + tire_model.PEY4 * ia_targ) * np.sign(sa))# now we get a coefficient for e_y
    b_y = kya / (d_y * c_y) # and then we can get b_y
    shy_0 = tire_model.PHY1 + tire_model.PHY2 * df_z
    svy_0 = (tire_model.PVY1 + tire_model.PVY2 * df_z) * fz_targ
    svy_g = (tire_model.PVY3 + tire_model.PVY4 * df_z) * fz_targ * ia_targ
    svy = svy_0 + svy_g
    kyg = (tire_model.PKY6 + tire_model.PKY7 * df_z) * fz_targ
    shy_g = (kyg * ia_targ - svy_g) / kya
    shy = shy_0 + shy_g
    sa_y = sa + shy
    fy = d_y * np.sin(c_y * np.arctan(b_y * sa_y - e_y * (b_y * sa_y - np.arctan(b_y * sa_y)))) + svy
    if offsets:
        return (fy, np.ones(1) * np.rad2deg(-shy), np.ones(1) * svy)
    return fy
fig = go.Figure()
for fza in sweeps:
    for iaa in [0, 2, 4]:
        plot_fit(fza, model, slip_angles, np.deg2rad(iaa), 0.0, 0.0, fig, df, f"Fit {fza} N {iaa} deg")
fig.update_layout(template="plotly_dark", title=f"Fy vs SA", xaxis_title="SA (deg)", yaxis_title="Fy (N)")
fig.update_yaxes(range=[-3000, 3000])
fig.show()

That did what I would describe as *fuck all* so lets move on to some camber dependent stiffness terms <br>
$K_{y\alpha} = P_{Ky1} * F_{z0} * sin(P_{Ky4} * arctan(\frac{F_z}{(P_{Ky2} + P_{Ky5}*\gamma^2) * F_{z0}}))(1-P_{Ky3}|\gamma|)$

In [ ]:
df = remove_time_gaps(filter_vel(filter_press(cornering, press=82700, d_press=6000), 11.1, 0.1))
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
def model(fz_targ, sa, ia_targ, press, kappa, offsets=False):
    df_z = (fz_targ - fz_nom) / fz_nom
    mu_y = (tire_model.PDY1 + tire_model.PDY2 * df_z) * (1 - tire_model.PDY3 * ia_targ**2)
    kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(fz_targ/((tire_model.PKY2 + tire_model.PKY5*(ia_targ**2))*fz_nom))) * (1 - tire_model.PKY3 * np.abs(ia_targ))
    c_y = tire_model.PCY1 # now we get a coefficient for c_y
    d_y = mu_y * fz_targ
    e_y = (tire_model.PEY1 + tire_model.PEY2 * df_z) * (1 + (tire_model.PEY5 * ia_targ**2) - (tire_model.PEY3 + tire_model.PEY4 * ia_targ) * np.sign(sa))# now we get a coefficient for e_y
    b_y = kya / (d_y * c_y) # and then we can get b_y
    shy_0 = tire_model.PHY1 + tire_model.PHY2 * df_z
    svy_0 = (tire_model.PVY1 + tire_model.PVY2 * df_z) * fz_targ
    svy_g = (tire_model.PVY3 + tire_model.PVY4 * df_z) * fz_targ * ia_targ
    svy = svy_0 + svy_g
    kyg = (tire_model.PKY6 + tire_model.PKY7 * df_z) * fz_targ
    shy_g = (kyg * ia_targ - svy_g) / kya
    shy = shy_0 + shy_g
    sa_y = sa + shy
    fy = d_y * np.sin(c_y * np.arctan(b_y * sa_y - e_y * (b_y * sa_y - np.arctan(b_y * sa_y)))) + svy
    if offsets:
        return (fy, np.ones(1) * np.rad2deg(-shy), np.ones(1) * svy)
    return fy
fig = go.Figure()
for fza in sweeps:
    for iaa in [0, 2, 4]:
        plot_fit(fza, model, slip_angles, np.deg2rad(iaa), 0.0, 0.0, fig, df, f"Fit {fza} N {iaa} deg")
fig.update_layout(template="plotly_dark", title=f"Fy vs SA", xaxis_title="SA (deg)", yaxis_title="Fy (N)")
fig.update_yaxes(range=[-3000, 3000])
fig.show()

Well that made things worse, but whatever, thats all the camber dependent terms, lets move on to pressure dependent terms in the next notebook<br>